In [1]:
from bson.objectid import ObjectId
from bson.son import SON
from datetime import datetime
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from pymongo import MongoClient
import re
from sshtunnel import SSHTunnelForwarder

# Connect to MongoDB

## Connection to remote instance

In [2]:
"""
with open('../keys.json') as k:
    keys = json.loads(k.read())
    MONGO_HOST = keys["SSH"]["MONGO_HOST"] # i.e. 10.9.13.14
    MONGO_DB = keys["SSH"]["MONGO_DB"] # i.e. dm_project 
    MONGO_USER = keys["SSH"]["MONGO_USER"] # i.e. studente
    MONGO_PASS = keys["SSH"]["MONGO_PASS"] # i.e. la password della vm
server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)
server.start() # remember to stop

client = MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]
db.list_collection_names()"""

## Local instance

In [ ]:
client = pymongo.MongoClient("localhost", 27017)
db = client["dm_project"]
db.list_collection_names()

#### Count of documents in each collection

Twitch:

In [5]:
db.twitch.count()

7938

Twitter:

In [6]:
db.twitter.count()

80909

#### Sizes of collections

Twitch:

In [7]:
print(db.command("collstats", 'twitch')['size']  / 1024 / 1024, 'MB')

7164.0113525390625 MB


Twitter:

In [8]:
print(db.command("collstats", 'twitter')['size'] / 1024 / 1024, 'MB')

36.78813171386719 MB


# Queries on Twitch Collection

We can load the list of games we are monitoring, in case we want to restrict our queries to these games:

In [9]:
with open("../top_50_games_to_monitor.json", "r") as f:
    game_list = json.load(f)
    top_games = game_list["selected_top_50_games"]["name"]

top_games[:5]

['Fortnite',
 'Call of Duty: Black Ops II',
 'Grand Theft Auto: San Andreas',
 'Clam Man',
 'Wii Play']

### Average daily views, channels and popularity for each game in the top_games list

The following query returns the average daily viewers, channels and popularity for each game in the list of monitored games

In [10]:
# MongoDB version >= 4

# pipeline = [{"$unwind": "$data"},
#             {"$match" : { "data.game_name" : { "$in" : top_games },  }},
#             { "$group": {
#                 "_id": {"name": "$data.game_name",
#                         "day": { "$dayOfMonth": {"$toDate" : "$timestamp" }},
#                         "month": { "$month": {"$toDate" : "$timestamp" }},
#                         "year": { "$year": {"$toDate" : "$timestamp" }} },
#                 "avgViewers": { "$avg": '$data.viewers'},
#                            "avgChannels": { "$avg": '$data.channels'},
#                            "avgPopularity": { "$avg": '$data.game_popularity'}
#             }},
#             {"$sort": {"_id": 1} }]

In [11]:
twitch_daily_avg_pipe = [{"$unwind": "$data"},
                        {"$match" : { "data.game_name" : { "$in" : top_games },  }},
                        { "$group": {
                            "_id": {"name": "$data.game_name",
                                    "norm_name": "$data.game_norm_name",
                                    "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, 
                                    "month" : { "$substr" : ["$timestamp", 5, 2 ] },     
                                    "day" :   { "$substr" : ["$timestamp", 8, 2 ] } },
                            "avgViewers": { "$avg": '$data.viewers'},
                            "avgChannels": { "$avg": '$data.channels'},
                            "avgPopularity": { "$avg": '$data.game_popularity'}
                            }},
                        {"$sort": {"_id": 1} }]

In [12]:
twitch_daily_avg = list(db.twitch.aggregate(twitch_daily_avg_pipe))

In [13]:
twitch_daily_avg[:5]

[{'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   'year': '2019',
   'month': '06',
   'day': '13'},
  'avgViewers': 17491.647058823528,
  'avgChannels': 1434.0761245674742,
  'avgPopularity': 17115.401384083045},
 {'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   'year': '2019',
   'month': '06',
   'day': '14'},
  'avgViewers': 15606.502083333333,
  'avgChannels': 1634.5229166666666,
  'avgPopularity': 15777.739583333334},
 {'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   'year': '2019',
   'month': '06',
   'day': '15'},
  'avgViewers': 15781.329166666666,
  'avgChannels': 1643.1083333333333,
  'avgPopularity': 15442.404166666667},
 {'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   'year': '2019',
   'month': '06',
   'day': '16'},
  'avgViewers': 18600.82463465553,
  'avgChannels': 1656.6492693110647,
  'avgPopularity': 18946.350730688937},
 {'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   

The following are the games initially chosen as top games, but that have not been collected again on twitch.

In [14]:
set(top_games) - set([elem['_id']['name'] for elem in twitch_daily_avg])

{'Clam Man'}

It is easier to visualize the result of the query as a dataframe:

In [15]:
twitch_stats_df = json_normalize(twitch_daily_avg)

In [16]:
twitch_stats_df = twitch_stats_df.rename(index=str,
                                         columns={"_id.name": "name", "_id.norm_name": "norm_name",
                                                  "_id.day": "day", "_id.month": "month","_id.year": "year"})
twitch_stats_df = twitch_stats_df[['name', "norm_name", 'day', 'month', 'year', 'avgViewers', 'avgPopularity', 'avgChannels']]

In [17]:
twitch_stats_df.head()

name     norm_name day month  year    avgViewers  avgPopularity  \
0  Apex Legends  apex legends  13    06  2019  17491.647059   17115.401384   
1  Apex Legends  apex legends  14    06  2019  15606.502083   15777.739583   
2  Apex Legends  apex legends  15    06  2019  15781.329167   15442.404167   
3  Apex Legends  apex legends  16    06  2019  18600.824635   18946.350731   
4  Apex Legends  apex legends  17    06  2019  19137.743750   18741.535417   

   avgChannels  
0  1434.076125  
1  1634.522917  
2  1643.108333  
3  1656.649269  
4  1555.712500

### Average daily views, channels and popularity for every game streamed on twitch

We can also query all games, not just those in the list:

In [18]:
all_twitch_daily_avg_pipe = [{"$unwind": "$data"},
                        { "$group": {
                            "_id": {"name": "$data.game_name",
                                    "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, 
                                    "month" : { "$substr" : ["$timestamp", 5, 2 ] },     
                                    "day" :   { "$substr" : ["$timestamp", 8, 2 ] } },
                            "avgViewers": { "$avg": '$data.viewers'},
                            "avgChannels": { "$avg": '$data.channels'},
                            "avgPopularity": { "$avg": '$data.game_popularity'}
                            }},
                        {"$sort": {"_id": 1} }]

In [19]:
all_twitch_daily_avg = list(db.twitch.aggregate(all_twitch_daily_avg_pipe))

In [20]:
all_twitch_daily_avg[:5]

[{'_id': {'name': ' 8 To Glory - The Official Game of the PBR',
   'year': '2019',
   'month': '06',
   'day': '14'},
  'avgViewers': 1.0,
  'avgChannels': 1.0,
  'avgPopularity': 0.0},
 {'_id': {'name': ' 8 To Glory - The Official Game of the PBR',
   'year': '2019',
   'month': '06',
   'day': '22'},
  'avgViewers': 5.0,
  'avgChannels': 1.0,
  'avgPopularity': 0.0},
 {'_id': {'name': ' F1 2019', 'year': '2019', 'month': '06', 'day': '20'},
  'avgViewers': 22.821428571428573,
  'avgChannels': 1.0,
  'avgPopularity': 20.0},
 {'_id': {'name': ' F1 2019', 'year': '2019', 'month': '06', 'day': '21'},
  'avgViewers': 481.517571884984,
  'avgChannels': 4.696485623003195,
  'avgPopularity': 482.17891373801916},
 {'_id': {'name': ' F1 2019', 'year': '2019', 'month': '06', 'day': '22'},
  'avgViewers': 207.31914893617022,
  'avgChannels': 2.5379939209726445,
  'avgPopularity': 201.72036474164133}]

In [21]:
all_twitch_daily_df = json_normalize(all_twitch_daily_avg)

In [22]:
all_twitch_daily_df = all_twitch_daily_df.rename(index=str,
                                         columns={"_id.name": "name", "_id.day": "day",
                                                  "_id.month": "month","_id.year": "year"})
all_twitch_daily_df = all_twitch_daily_df[['name', 'day', 'month', 'year', 'avgViewers', 'avgPopularity', 'avgChannels']]

In [23]:
all_twitch_daily_df.head()

name day month  year  avgViewers  \
0   8 To Glory - The Official Game of the PBR  14    06  2019    1.000000   
1   8 To Glory - The Official Game of the PBR  22    06  2019    5.000000   
2                                     F1 2019  20    06  2019   22.821429   
3                                     F1 2019  21    06  2019  481.517572   
4                                     F1 2019  22    06  2019  207.319149   

   avgPopularity  avgChannels  
0       0.000000     1.000000  
1       0.000000     1.000000  
2      20.000000     1.000000  
3     482.178914     4.696486  
4     201.720365     2.537994

We add the column 'daily_top', which is the game's rank in the ordered list of daily most spectated games on average (with respect to every game streamed on Twitch).

In [24]:
all_twitch_daily_df['daily_top'] = all_twitch_daily_df.sort_values(['day', 'month', 'year', 'avgViewers'], 
                                                           ascending=[True, True, True, False]).groupby(['day', 'month', 'year']).cumcount()
all_twitch_daily_df['daily_top'] = all_twitch_daily_df['daily_top'] + 1

In [25]:
all_twitch_daily_df.sort_values(['day', 'month', 'year', 'avgViewers'], ascending=[True, True, True, False]).head()

name day month  year     avgViewers  avgPopularity  \
42691   League of Legends  13    06  2019  139492.283737  140292.190311   
29121            Fortnite  13    06  2019  117869.685121  116048.532872   
32570  Grand Theft Auto V  13    06  2019   97704.619377   95741.335640   
39449       Just Chatting  13    06  2019   97585.868512   97702.740484   
21213              Dota 2  13    06  2019   69030.173010   69218.449827   

       avgChannels  daily_top  
42691  2966.896194          1  
29121  7186.041522          2  
32570  1057.024221          3  
39449  1351.307958          4  
21213   756.505190          5

In [26]:
all_twitch_daily_df[all_twitch_daily_df['name'] == 'Tetris']

name day month  year  avgViewers  avgPopularity  avgChannels  \
79065  Tetris  13    06  2019   49.739130      48.155797     6.847826   
79066  Tetris  14    06  2019   35.179104      35.605544     7.695096   
79067  Tetris  15    06  2019   31.952070      32.525054     6.893246   
79068  Tetris  16    06  2019   51.653595      47.714597     6.472767   
79069  Tetris  17    06  2019   59.634409      65.215054     5.600000   
79070  Tetris  18    06  2019   54.096280      57.597374     6.601751   
79071  Tetris  19    06  2019   66.323913      63.771739     5.093478   
79072  Tetris  20    06  2019   29.947712      29.187364     6.485839   
79073  Tetris  21    06  2019   53.063596      53.901316     6.050439   
79074  Tetris  22    06  2019   44.438053      40.663717     5.473451   
79075  Tetris  23    06  2019   29.812785      30.933790     4.988584   
79076  Tetris  24    06  2019   34.866370      32.467706     5.928731   
79077  Tetris  25    06  2019   55.900000      59.262222     5.486667   
79078  Tetris  26    06  2019   49.111374      46.931280     4.864929   
79079  Tetris  27    06  2019   50.995050      55.044554     4.599010   

       daily_top  
79065        691  
79066        877  
79067        972  
79068        757  
79069        656  
79070        712  
79071        615  
79072        946  
79073        682  
79074        808  
79075        979  
79076        826  
79077        636  
79078        683  
79079        531

### Daily statistics for a single game

In [27]:
mario_daily_pipe = [{"$unwind": "$data"},
                    {"$match" : { "data.game_name" : "Super Mario Bros."  }},
                    { "$group": {
                        "_id": {"name": "$data.game_name",
                                "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, 
                                "month" : { "$substr" : ["$timestamp", 5, 2 ] },     
                                "day" :   { "$substr" : ["$timestamp", 8, 2 ] } },
                        "avgViewers": { "$avg": '$data.viewers'},
                        "avgChannels": { "$avg": '$data.channels'},
                        "avgPopularity": { "$avg": '$data.game_popularity'}
                        }},
                    {"$sort": {"_id": 1} }]

In [28]:
mario_daily = list(db.twitch.aggregate(mario_daily_pipe))

In [29]:
mario_daily[:3]

[{'_id': {'name': 'Super Mario Bros.',
   'year': '2019',
   'month': '06',
   'day': '13'},
  'avgViewers': 51.2027027027027,
  'avgChannels': 3.4414414414414414,
  'avgPopularity': 44.531531531531535},
 {'_id': {'name': 'Super Mario Bros.',
   'year': '2019',
   'month': '06',
   'day': '14'},
  'avgViewers': 13.919014084507042,
  'avgChannels': 2.9119718309859155,
  'avgPopularity': 12.535211267605634},
 {'_id': {'name': 'Super Mario Bros.',
   'year': '2019',
   'month': '06',
   'day': '15'},
  'avgViewers': 74.43236714975845,
  'avgChannels': 3.7028985507246377,
  'avgPopularity': 72.28985507246377}]

### Every document regarding a single game

In [30]:
mario_all = list(db.twitch.find({'data.game_name': 'Super Mario Bros.'},
               {'timestamp':1, 'data.$': 1}))

In [31]:
mario_all[0:3]

[{'_id': ObjectId('5d0f44df7c6c727793bcd374'),
  'data': [{'game_giantbomb_id': 15544,
    'channels': 2,
    'game_popularity': 3,
    'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Super%20Mario%20Bros.-240x144.jpg',
    'game__id': 509508,
    'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Super%20Mario%20Bros.-272x380.jpg',
    'game_norm_name': 'super mario bros.',
    'game_name': 'Super Mario Bros.',
    'viewers': 2}],
  'timestamp': '2019-06-23 09:22:34.124005'},
 {'_id': ObjectId('5d0f45937c6c727793bcd375'),
  'data': [{'game_giantbomb_id': 15544,
    'channels': 2,
    'game_popularity': 2,
    'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Super%20Mario%20Bros.-240x144.jpg',
    'game__id': 509508,
    'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Super%20Mario%20Bros.-272x380.jpg',
    'game_norm_name': 'super mario bros.',
    'game_name': 'Super Mario Bros.',
    'viewers': 3}],
  'timestamp': '2019-06-23 09:25:34.0781

### Query on a single game and day

We can compile a regex in order to find all documents regarding, for instance, 20/06/2019.

In [32]:
day = re.compile("^2019-06-20 \d+:\d+:\d+.\d+$")

In [33]:
mario_0620 = list(db.twitch.find({'data.game_name': 'Super Mario Bros.',
                     'timestamp':  {"$regex" : day  }},
                    {'timestamp':1, 'data.$': 1}))

In [34]:
mario_0620[:3]

[{'_id': ObjectId('5d0f4d10f0bd064ad83b1f8d'),
  'timestamp': '2019-06-20 00:02:34.009747',
  'data': [{'channels': 18,
    'game__id': 509508,
    'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Super%20Mario%20Bros.-272x380.jpg',
    'game_giantbomb_id': 15544,
    'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Super%20Mario%20Bros.-240x144.jpg',
    'game_name': 'Super Mario Bros.',
    'game_norm_name': 'super mario bros.',
    'game_popularity': 416,
    'viewers': 366}]},
 {'_id': ObjectId('5d0f4d11f0bd064ad83b1f8e'),
  'timestamp': '2019-06-20 00:05:34.004106',
  'data': [{'channels': 19,
    'game__id': 509508,
    'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Super%20Mario%20Bros.-272x380.jpg',
    'game_giantbomb_id': 15544,
    'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Super%20Mario%20Bros.-240x144.jpg',
    'game_name': 'Super Mario Bros.',
    'game_norm_name': 'super mario bros.',
    'game_popularity': 416,
    'vi

### Statistics for a single day

In [35]:
top_twitch_0620_pipe = [{"$unwind": "$data"},
                        {"$match" : {"$and":
                                     [{ "data.game_name" : { "$in" : top_games }  },
                                      {'timestamp':  {"$regex" : day}}] }},
                        { "$group": {"_id":
                                     {"name": "$data.game_name"},
                                     "avgViewers": { "$avg": '$data.viewers'},
                                     "avgChannels": { "$avg": '$data.channels'},
                                     "avgPopularity": { "$avg": '$data.game_popularity'}
                                     }},
                        {"$sort": {"_id": 1} }]

In [36]:
top_twitch_0620 = list(db.twitch.aggregate(top_twitch_0620_pipe))

In [37]:
top_twitch_0620[:5]

[{'_id': {'name': 'Apex Legends'},
  'avgViewers': 15973.658333333333,
  'avgChannels': 1600.80625,
  'avgPopularity': 16233.808333333332},
 {'_id': {'name': 'Auto Chess'},
  'avgViewers': 3473.3145833333333,
  'avgChannels': 102.01666666666667,
  'avgPopularity': 3548.9583333333335},
 {'_id': {'name': 'Call of Duty: Advanced Warfare'},
  'avgViewers': 8.268436578171091,
  'avgChannels': 3.017699115044248,
  'avgPopularity': 7.899705014749262},
 {'_id': {'name': 'Call of Duty: Black Ops'},
  'avgViewers': 81.78646934460888,
  'avgChannels': 12.930232558139535,
  'avgPopularity': 83.1416490486258},
 {'_id': {'name': 'Call of Duty: Black Ops 4'},
  'avgViewers': 10145.0125,
  'avgChannels': 1122.56875,
  'avgPopularity': 10217.6875}]

### Game with highest average viewers for each day

In [38]:
highest_avgView_per_day_pipe = [
    {"$unwind": "$data"},
    { "$group": {
                "_id": {"name": "$data.game_name",
                        "day" :   { "$substr" : ["$timestamp", 8, 2 ] },
                        "month" : { "$substr" : ["$timestamp", 5, 2 ] },         
                        "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, },                        
                "avgViewers": { "$avg": '$data.viewers'}
            }},
    {"$sort": {"avgViewers": -1}},
    {"$group": {"_id": {
        "day": "$_id.day",
        "month": "$_id.month",
        "year": "$_id.year" },    
                "max_views": {"$first": "$avgViewers"},
                "game": {"$first": "$_id.name"}}},
    {"$project": {
            "_id": 0,
            "day": "$_id.day",
            "month": "$_id.month",
            "year": "$_id.year",
            "max_views": 1,
            "game": 1} },
    {"$sort": {"day": 1}}]

In [39]:
highest_avgView_per_day = list(db.twitch.aggregate(highest_avgView_per_day_pipe))

In [40]:
highest_avgView_per_day[:5]

[{'max_views': 139492.28373702423,
  'game': 'League of Legends',
  'day': '13',
  'month': '06',
  'year': '2019'},
 {'max_views': 138835.44166666668,
  'game': 'League of Legends',
  'day': '14',
  'month': '06',
  'year': '2019'},
 {'max_views': 179458.00625,
  'game': 'League of Legends',
  'day': '15',
  'month': '06',
  'year': '2019'},
 {'max_views': 165168.76617954072,
  'game': 'League of Legends',
  'day': '16',
  'month': '06',
  'year': '2019'},
 {'max_views': 131107.91666666666,
  'game': 'Fortnite',
  'day': '17',
  'month': '06',
  'year': '2019'}]

We could write a very similar query to find, for each day, the game with the highest average popularity or number of channels.

# Queries on Twitter

In [44]:
with open("../top_50_games_to_monitor.json", "r") as f:
    game_list = json.load(f)
    tweet_top_games = game_list["selected_top_50_games"]["norm_name"]

tweet_top_games[:5]

['death stranding',
 'new super mario bros. wii',
 'pokémon red/blue',
 'mario kart ds',
 'grand theft auto iv']

### Daily total retweets, likes, and tweet count for each game in the top_games list

In [45]:
tweet_daily_pipe = [{"$match" : { "query" : { "$in" : tweet_top_games},  }},
                    { "$group": {"_id": {"name": "$query",
                                         "day": { "$dayOfMonth": "$date" },
                                         "month": { "$month": "$date" },
                                         "year": { "$year": "$date" }},
                                 "tot_retweets": { "$sum": "$retweets" },
                                 "tot_likes": { "$sum" : "$likes"},
                                 "tweet_count" : { "$sum": 1 }
                                 }},
                    {"$sort": {"_id": 1} }]

In [46]:
tweet_daily = list(db.twitter.aggregate(tweet_daily_pipe))

In [47]:
tweet_daily[0:4]

[{'_id': {'name': 'apex legends', 'day': 13, 'month': 6, 'year': 2019},
  'tot_retweets': 54,
  'tot_likes': 349,
  'tweet_count': 96},
 {'_id': {'name': 'apex legends', 'day': 14, 'month': 6, 'year': 2019},
  'tot_retweets': 140,
  'tot_likes': 1022,
  'tweet_count': 200},
 {'_id': {'name': 'apex legends', 'day': 15, 'month': 6, 'year': 2019},
  'tot_retweets': 160,
  'tot_likes': 1154,
  'tweet_count': 200},
 {'_id': {'name': 'apex legends', 'day': 16, 'month': 6, 'year': 2019},
  'tot_retweets': 173,
  'tot_likes': 1491,
  'tweet_count': 169}]

In [48]:
tweet_daily_df = json_normalize(tweet_daily)
tweet_daily_df = tweet_daily_df.rename(index=str,
                                       columns={"_id.name": "name", "_id.day": "day","_id.month": "month","_id.year": "year"})
tweet_daily_df = tweet_daily_df[['name', 'day', 'month', 'year', 'tot_retweets', 'tot_likes', 'tweet_count']]
tweet_daily_df.head()

name  day  month  year  tot_retweets  tot_likes  tweet_count
0  apex legends   13      6  2019            54        349           96
1  apex legends   14      6  2019           140       1022          200
2  apex legends   15      6  2019           160       1154          200
3  apex legends   16      6  2019           173       1491          169
4  apex legends   17      6  2019           112        692           16

In [49]:
tweet_daily_df[tweet_daily_df['name'] == 'tetris']

name  day  month  year  tot_retweets  tot_likes  tweet_count
536  tetris   13      6  2019            28        282          200
537  tetris   14      6  2019           726       4310           76
538  tetris   15      6  2019            60        597          200
539  tetris   16      6  2019           143        903          200
540  tetris   17      6  2019           175        830          200
541  tetris   18      6  2019           360       1830          110
542  tetris   19      6  2019           317       1547          200
543  tetris   20      6  2019           764      11352          200
544  tetris   21      6  2019           188       1452          200
545  tetris   22      6  2019            38        305           29
546  tetris   23      6  2019           125        791          200
547  tetris   24      6  2019           112       1456          200
548  tetris   25      6  2019            90       1417           90
549  tetris   26      6  2019            44        585          200

### Game with the highest retweet count for each day:

In [50]:
pipeline_highest_retweet_per_day = [{
    "$group": {"_id": {"name": "$query",
                       "day": {"$dayOfMonth": "$date"},
                       "month": {"$month": "$date"},
                       "year": {"$year": "$date"} },
               "tot_retweets": { "$sum": "$retweets"}} },
    {"$sort": {"tot_retweets": -1} },
    {"$group": {"_id": {"day": "$_id.day",
                        "month": "$_id.month",
                        "year": "$_id.year"},
                "max_retweets": {"$first": "$tot_retweets"},
                "game": {"$first": "$_id.name"} } },
    {"$project": {"_id": 0,
                  "day": "$_id.day",
                  "month": "$_id.month",
                  "year": "$_id.year",
                  "max_retweets": 1,
                  "game": 1 } },
    {"$sort": { "day": 1}}]

In [51]:
highest_retweet_per_day = list(db.twitter.aggregate(pipeline_highest_retweet_per_day))
highest_retweet_per_day[:5]

[{'max_retweets': 419,
  'game': 'fortnite',
  'day': 13,
  'month': 6,
  'year': 2019},
 {'max_retweets': 1878,
  'game': 'fortnite',
  'day': 14,
  'month': 6,
  'year': 2019},
 {'max_retweets': 9617,
  'game': 'pokémon gold/silver',
  'day': 15,
  'month': 6,
  'year': 2019},
 {'max_retweets': 3365, 'game': 'slots', 'day': 16, 'month': 6, 'year': 2019},
 {'max_retweets': 6082,
  'game': 'overwatch',
  'day': 17,
  'month': 6,
  'year': 2019}]

### Game with the highest likes for each day:

In [52]:
pipeline_highest_likes_per_day = [{
    "$group": {"_id": {"name": "$query",
                       "day": {"$dayOfMonth": "$date"},
                       "month": {"$month": "$date"},
                       "year": {"$year": "$date"} },
               "tot_likes": { "$sum": "$likes"}} },
    {"$sort": {"tot_likes": -1} },
    {"$group": {"_id": {"day": "$_id.day",
                        "month": "$_id.month",
                        "year": "$_id.year"},
                "max_likes": {"$first": "$tot_likes"},
                "game": {"$first": "$_id.name"} } },
    {"$project": {"_id": 0,
                  "day": "$_id.day",
                  "month": "$_id.month",
                  "year": "$_id.year",
                  "max_likes": 1,
                  "game": 1 } },
    {"$sort": { "day": 1}}]

In [53]:
highest_likes_per_day = list(db.twitter.aggregate(pipeline_highest_likes_per_day))
highest_likes_per_day

[{'max_likes': 7344, 'game': 'fortnite', 'day': 13, 'month': 6, 'year': 2019},
 {'max_likes': 34285, 'game': 'fortnite', 'day': 14, 'month': 6, 'year': 2019},
 {'max_likes': 48678,
  'game': 'pokémon gold/silver',
  'day': 15,
  'month': 6,
  'year': 2019},
 {'max_likes': 33085, 'game': 'fortnite', 'day': 16, 'month': 6, 'year': 2019},
 {'max_likes': 35726, 'game': 'fortnite', 'day': 17, 'month': 6, 'year': 2019},
 {'max_likes': 53061, 'game': 'fortnite', 'day': 18, 'month': 6, 'year': 2019},
 {'max_likes': 15202, 'game': 'fortnite', 'day': 19, 'month': 6, 'year': 2019},
 {'max_likes': 30358, 'game': 'fortnite', 'day': 20, 'month': 6, 'year': 2019},
 {'max_likes': 22664, 'game': 'fortnite', 'day': 21, 'month': 6, 'year': 2019},
 {'max_likes': 43675,
  'game': 'minecraft',
  'day': 22,
  'month': 6,
  'year': 2019},
 {'max_likes': 25375,
  'game': 'minecraft',
  'day': 23,
  'month': 6,
  'year': 2019},
 {'max_likes': 39009, 'game': 'fortnite', 'day': 24, 'month': 6, 'year': 2019},
 {'m

### Which is the day with the highest likes? And which is the one with the lowest values?

In [54]:
pipeline_highest_likes = [{
    "$group": {"_id": {"day": {"$dayOfMonth": "$date"},
                       "month": {"$month": "$date"},
                       "year": {"$year": "$date"} },
               "tot_likes": { "$sum": "$likes"}} },
    {"$sort": {"tot_likes": -1} },
    {"$group": {"_id": {"day": "$_id.day",
                        "month": "$_id.month",
                        "year": "$_id.year"},
                "max_likes": {"$first": "$tot_likes"}}},
    {"$project": {"_id": 0,
                  "day": "$_id.day",
                  "month": "$_id.month",
                  "year": "$_id.year",
                  "max_likes": 1} },
    {"$sort": { "max_likes": -1}}]

In [55]:
highest_likes = list(db.twitter.aggregate(pipeline_highest_likes))
print('Day with highest likes: \n\t', highest_likes[0],
      '\n\nDay with lowest likes: \n\t', highest_likes[-1])

Day with highest likes: 
	 {'max_likes': 129262, 'day': 15, 'month': 6, 'year': 2019} 

Day with lowest likes: 
	 {'max_likes': 22493, 'day': 13, 'month': 6, 'year': 2019}


# Total "buzz"

We are interested in using both Twitch and Twitter data for determing which game has the highest "buzz", that is for us the weighted sum of the standardized values of the average views on Twitch and the total retweets and likes on Twitter.

In [56]:
pip = [{"$match" : { "query" : { "$in" : tweet_top_games},  }},
       {"$group": {"_id": {"name": "$query",
                          "day": {"$dayOfMonth": "$date"},
                          "month": {"$month": "$date"},
                          "year": {"$year": "$date"}},
        "total_retweets": {"$sum": "$retweets"},
        "total_likes": {"$sum": "$likes"}}},
       {"$project": {"_id": 0,
                     "norm_name": "$_id.name",
                     "day": "$_id.day",
                     "month": "$_id.month",
                     "year": "$_id.year",
                     "total_retweets": 1,
                     "total_likes": 1}},
       {"$sort": {"day": 1}}]

In [57]:
tweet_stats_query_results = list(db.twitter.aggregate(pip))
tweet_stats_df = pd.DataFrame.from_records(tweet_stats_query_results)
tweet_stats_df.head()

day  month                      norm_name  total_likes  total_retweets  \
0   13      6            pokémon gold/silver           16               1   
1   13      6                    hearthstone          588              37   
2   13      6   grand theft auto san andreas            1               2   
3   13      6                       fortnite         7344             419   
4   13      6  playerunknown's battlegrounds           20               4   

   year  
0  2019  
1  2019  
2  2019  
3  2019  
4  2019

In [58]:
server.stop()

In [59]:
twitch_stats_df.head()

name     norm_name day month  year    avgViewers  avgPopularity  \
0  Apex Legends  apex legends  13    06  2019  17491.647059   17115.401384   
1  Apex Legends  apex legends  14    06  2019  15606.502083   15777.739583   
2  Apex Legends  apex legends  15    06  2019  15781.329167   15442.404167   
3  Apex Legends  apex legends  16    06  2019  18600.824635   18946.350731   
4  Apex Legends  apex legends  17    06  2019  19137.743750   18741.535417   

   avgChannels  
0  1434.076125  
1  1634.522917  
2  1643.108333  
3  1656.649269  
4  1555.712500

**Let's join *tweet_stats_df* and *twitch_stats_df***

In [60]:
twitch_stats_df.day = twitch_stats_df.day.astype(int)
twitch_stats_df.month = twitch_stats_df.month.astype(int)
twitch_stats_df.year = twitch_stats_df.year.astype(int)

In [61]:
combined_stats = pd.merge(tweet_stats_df, twitch_stats_df, on = ['norm_name', 'day', 'month', 'year'], how="outer")

# replace some twitch and twitter missing values with 0: if the value is missing, the game was not streamed or tweeted
idx = ['total_retweets', 'total_likes','avgViewers', 'avgPopularity', 'avgChannels']
combined_stats[idx] = combined_stats[idx].fillna(0)

combined_stats.head()

day  month                      norm_name  total_likes  total_retweets  \
0   13      6            pokémon gold/silver         16.0             1.0   
1   13      6                    hearthstone        588.0            37.0   
2   13      6   grand theft auto san andreas          1.0             2.0   
3   13      6                       fortnite       7344.0           419.0   
4   13      6  playerunknown's battlegrounds         20.0             4.0   

   year                           name     avgViewers  avgPopularity  \
0  2019            Pokémon Gold/Silver       8.733813       7.705036   
1  2019                    Hearthstone   21380.671280   21350.159170   
2  2019  Grand Theft Auto: San Andreas     455.910035     436.757785   
3  2019                       Fortnite  117869.685121  116048.532872   
4  2019  PLAYERUNKNOWN'S BATTLEGROUNDS   40519.429066   40224.128028   

   avgChannels  
0     1.942446  
1   225.629758  
2    28.186851  
3  7186.041522  
4  1299.048443

### Computing total buzz

In [62]:
rtw_mu = np.mean(combined_stats['total_retweets'])
rtw_sd = np.std(combined_stats['total_retweets'])
combined_stats['std_tot_rtw'] = (combined_stats['total_retweets'] - rtw_mu) / rtw_sd

like_mu = np.mean(combined_stats['total_likes'])
like_sd = np.std(combined_stats['total_likes'])

combined_stats['std_tot_likes'] = (combined_stats['total_likes'] - like_mu) / like_sd

vw_mu = np.mean(combined_stats['avgViewers'])
vw_sd = np.std(combined_stats['avgViewers'])

combined_stats['std_avgViewers'] = (combined_stats['avgViewers'] - vw_mu) / vw_sd

Now that the features are standardized, we combine them in order to create the buzz feature. Since data from twitch and twitter should have the same weight, we create the new feature as a weighted sum.

In [63]:
weight_rtw = 0.25
weight_likes = 0.25
weight_twitch = 0.5

In [64]:
combined_stats['buzz'] = (weight_likes*combined_stats.std_tot_likes
                              + weight_rtw*combined_stats.std_tot_rtw
                              + weight_twitch*combined_stats.std_avgViewers)

In [65]:
combined_stats[['name','day', 'month', 'year', 'std_avgViewers', 'std_tot_likes', 'std_tot_rtw', 'buzz']].head()

name  day  month  year  std_avgViewers  \
0            Pokémon Gold/Silver   13      6  2019       -0.523800   
1                    Hearthstone   13      6  2019        0.183808   
2  Grand Theft Auto: San Andreas   13      6  2019       -0.508995   
3                       Fortnite   13      6  2019        3.378483   
4  PLAYERUNKNOWN'S BATTLEGROUNDS   13      6  2019        0.817477   

   std_tot_likes  std_tot_rtw      buzz  
0      -0.287895    -0.257687 -0.398296  
1      -0.185497    -0.212410 -0.007573  
2      -0.290580    -0.256429 -0.391250  
3       1.023938     0.268030  2.012233  
4      -0.287179    -0.253914  0.273465

## In which day a game obtained the highest "buzz"?

In [66]:
combined_stats.loc[combined_stats['buzz'].idxmax()]

day                     18
month                    6
norm_name         fortnite
total_likes          53061
total_retweets        8851
year                  2019
name              Fortnite
avgViewers          105128
avgPopularity       104166
avgChannels        7266.33
std_tot_rtw        10.8729
std_tot_likes      9.20803
std_avgViewers     2.95661
buzz               6.49854
Name: 232, dtype: object

## In which day a game obtained the lowest "buzz"?

In [67]:
combined_stats.loc[combined_stats['buzz'].idxmin()]

day                          13
month                         6
norm_name         mario kart ds
total_likes                   0
total_retweets                0
year                       2019
name                        NaN
avgViewers                    0
avgPopularity                 0
avgChannels                   0
std_tot_rtw           -0.258945
std_tot_likes         -0.290759
std_avgViewers        -0.524089
buzz                  -0.399471
Name: 36, dtype: object

## Which is the game with the highest cumulative "buzz"?

In [68]:
cumulative_buzz = combined_stats.groupby("norm_name")['buzz'].sum().reset_index()
cumulative_buzz.loc[cumulative_buzz['buzz'].idxmax()]

norm_name    fortnite
buzz          53.9799
Name: 17, dtype: object

## Which is the game with the lowest cumulative "buzz"?

In [69]:
cumulative_buzz.loc[cumulative_buzz['buzz'].idxmin()]

norm_name    call of duty modern warfare iii
buzz                                -5.98527
Name: 9, dtype: object

## Games sorted by cumulative buzz

In [70]:
cumulative_buzz.sort_values('buzz', ascending = False).reset_index()

index                        norm_name       buzz
0      17                         fortnite  53.979884
1      24                league of legends  26.325080
2      22                    just chatting  16.893850
3      20               grand theft auto v  14.463689
4      28                        minecraft  11.291719
5      33                        overwatch  10.586666
6      14                          dota ii   8.852364
7      37                            slots   8.396402
8      11  counter-strike global offensive   5.581982
9      49                world of warcraft   4.636942
10      0                     apex legends   2.075119
11     21                      hearthstone   1.778720
12     34    playerunknown's battlegrounds   1.621780
13     12                 dead by daylight   1.520694
14     35              pokémon gold/silver  -0.593629
15     44   tom clancy's rainbow six siege  -1.278021
16     25              magic the gathering  -2.340839
17      7        call of duty black ops iv  -2.625258
18      4           call of duty black ops  -3.020850
19     16                         fifa xix  -3.022834
20     39                super mario bros.  -3.349871
21     30          music & performing arts  -3.743271
22     42                           tetris  -3.746038
23      1                       auto chess  -3.851084
24     13                  death stranding  -3.873131
25     41            talk shows & podcasts  -4.213866
26      2              bio inc. redemption  -4.376539
27     40                super mario world  -4.512595
28     15                        duck hunt  -4.650868
29     47                       wii sports  -4.690403
30     26                    mario kart ds  -5.107132
31     46                         wii play  -5.135300
32     45         total war three kingdoms  -5.204597
33     10                         clam man  -5.497921
34     43       the elder scrolls v skyrim  -5.524472
35     48                wii sports resort  -5.571307
36     36                 pokémon red/blue  -5.712955
37     27                   mario kart wii  -5.740327
38     31            new super mario bros.  -5.744935
39      6       call of duty black ops iii  -5.787040
40      8              call of duty ghosts  -5.788901
41     19     grand theft auto san andreas  -5.857358
42     23                layers of fear ii  -5.861260
43      3    call of duty advanced warfare  -5.906256
44     38          splitgate arena warfare  -5.919611
45     18              grand theft auto iv  -5.933140
46      5        call of duty black ops ii  -5.938599
47     29                modern warfare ii  -5.939010
48     32        new super mario bros. wii  -5.960404
49      9  call of duty modern warfare iii  -5.985266

## Integration with VGChartz

We integrate the list of monitored games, sorted by their buzz, with the data downloaded from VGChartz.

The missing values are due to data not available on VGChartz for some games or for some Twitch streams.

In [71]:
vgsalesn = pd.read_csv('../dataset/vgsales_n.csv')

In [72]:
platform = vgsalesn.groupby('Norm_name')['Platform'].apply(', '.join)
cr_score = vgsalesn.groupby('Norm_name')['Critic_Score'].mean()
sales = vgsalesn.groupby('Norm_name')['Global_Sales'].sum()
u_score  = vgsalesn.groupby('Norm_name')['User_Score'].mean()
pub = vgsalesn.groupby('Norm_name')['Publisher'].apply(', '.join)
dev = vgsalesn.groupby('Norm_name')['Developer'].apply(', '.join)

In [73]:
vgsalesn = pd.DataFrame({'Norm_name' : sales.index, 'Global_Sales': sales,
                   'Platforms': platform,  'Critic_Score' : cr_score, 'User_Score' : u_score,
                  'Publisher' : pub, 'Developer' : dev})

vgsalesn = vgsalesn.reset_index(drop = 1)

In [74]:
merged = cumulative_buzz.merge(vgsalesn, left_on = 'norm_name', right_on = 'Norm_name', how = 'outer')
merged = merged[merged.norm_name.isin(tweet_top_games)].sort_values('buzz', ascending = False)
merged = merged.reset_index(drop = True).drop('Norm_name', 1)
merged['Global_Sales'] = merged['Global_Sales'].replace(0, np.nan)

In [75]:
merged

norm_name       buzz  Global_Sales  \
0                          fortnite  53.979884          2.37   
1                 league of legends  26.325080           NaN   
2                     just chatting  16.893850           NaN   
3                grand theft auto v  14.463689         65.62   
4                         minecraft  11.291719         33.83   
5                         overwatch  10.586666          8.18   
6                           dota ii   8.852364           NaN   
7                             slots   8.396402           NaN   
8   counter-strike global offensive   5.581982           NaN   
9                 world of warcraft   4.636942          6.35   
10                     apex legends   2.075119           NaN   
11                      hearthstone   1.778720           NaN   
12    playerunknown's battlegrounds   1.621780          2.50   
13                 dead by daylight   1.520694          0.34   
14              pokémon gold/silver  -0.593629         23.10   
15   tom clancy's rainbow six siege  -1.278021          7.37   
16              magic the gathering  -2.340839           NaN   
17        call of duty black ops iv  -2.625258           NaN   
18           call of duty black ops  -3.020850         30.99   
19                         fifa xix  -3.022834         12.22   
20                super mario bros.  -3.349871         40.24   
21          music & performing arts  -3.743271           NaN   
22                           tetris  -3.746038         35.84   
23                       auto chess  -3.851084           NaN   
24                  death stranding  -3.873131           NaN   
25            talk shows & podcasts  -4.213866           NaN   
26              bio inc. redemption  -4.376539           NaN   
27                super mario world  -4.512595         20.61   
28                        duck hunt  -4.650868         28.31   
29                       wii sports  -4.690403         82.65   
30                    mario kart ds  -5.107132         23.26   
31                         wii play  -5.135300         28.92   
32         total war three kingdoms  -5.204597           NaN   
33                         clam man  -5.497921           NaN   
34       the elder scrolls v skyrim  -5.524472         20.51   
35                wii sports resort  -5.571307         32.90   
36                 pokémon red/blue  -5.712955         31.37   
37                   mario kart wii  -5.740327         35.98   
38            new super mario bros.  -5.744935         29.85   
39       call of duty black ops iii  -5.787040         26.72   
40              call of duty ghosts  -5.788901         28.80   
41     grand theft auto san andreas  -5.857358         23.87   
42                layers of fear ii  -5.861260           NaN   
43    call of duty advanced warfare  -5.906256         21.78   
44          splitgate arena warfare  -5.919611           NaN   
45              grand theft auto iv  -5.933140         22.53   
46        call of duty black ops ii  -5.938599         29.59   
47                modern warfare ii  -5.939010         25.02   
48        new super mario bros. wii  -5.960404         28.51   
49  call of duty modern warfare iii  -5.985266         30.71   

                                            Platforms  Critic_Score  \
0                                   PS4, XOne, NS, PC      8.500000   
1                                             PC, OSX      8.000000   
2                                                 NaN           NaN   
3                            PS3, PS4, X360, PC, XOne      9.366667   
4   PC, X360, PS4, PS3, XOne, PSV, NS, WiiU, 3DS, ...      8.160000   
5                                       PS4, XOne, PC      9.000000   
6                                      PC, OSX, Linux           NaN   
7                                                2600           NaN   
8                                   PC, XBL, PSN, OSX           NaN   
9                                             PC, O